In [1]:
%matplotlib inline


# 使用PyTorch实现Chatbot




## 简介
本教程会介绍使用seq2seq模型实现一个chatbot，使用的训练数据来自[Cornell电影对话语料库](https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html)。本节中我们介绍使用seq2seq模型搭建一个聊天机器人。使用的训练数据来自康纳尔电影对话语料库。传统的对话系统要么基于检索的方法，提前准备一个问答库，根据用户的输入寻找类似的问题和答案。这样更像一个问答系统，它很难进行多轮的交互，而且答案是固定不变的。要么基于预先设置的对话流程，这主要用于填槽的任务，比如查询机票需要用户提供日期，达到城市等信息。这种方法的缺点是比较死板，如果用户的意图在设计的流程之外，那么就无法处理，而且对话的流程也一般比较固定，要支持用户随意的话题内跳转和话题间切换比较困难。
目前研究热点是根据大量的对话数据，通过深度学习技术，自动的端到端的使用Seq2Seq模型学习对话模型。它的好处是不需要人来设计这个对话流程，完全是数据驱动的方法。它的缺点是流程不受人(开发者)控制，在严肃的场景(比如客服)下使用会有比较大的风险，而且需要大量的对话数据，这在很多实际应用中是很难得到的。这点在我们给移动公司打客服电话的时候，就会发现明显的改变。以前我们是听到语音提示，按键进行选择，而如今更多的是，让你说出你的需求，客服会根据你说出的需求，进行选择和回答。
在本例中，我们将在PyTorch中实现这种模型，我们会分步骤具体讲解实现过程，具体实现代码会用灰色底纹标注。下面是具体实现后的对话效果示例：

```
> hello?
Bot: hello .
> where am I?
Bot: you re in a hospital .
> who are you?
Bot: i m a lawyer .
> how are you doing?
Bot: i m fine .
> are you my friend?
Bot: no .
> you're under arrest
Bot: i m trying to help you !
> i'm just kidding
Bot: i m sorry .
> where are you from?
Bot: san francisco .
> it's time for me to leave
Bot: i know .
> goodbye
Bot: goodbye .
```




## 准备

首先我们通过(http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip)下载训练语料库，这是一个zip文件，把它下载后到项目目录的子目录data下。接下来我们导入需要用到的模块，这主要是PyTorch的模块：


In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math


USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")


## 加载和预处理数据

接下来我们需要对原始数据进行变换然后用合适的数据结构加载到内存里。

[Cornell电影对话语料库](https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html)是电影人物的对话数据，它包括：

-  10,292对电影人物(一部电影有多个人物，他们两两之间可能存在对话)的220,579个对话
-  617部电影的9,035个人物
-  总共304,713个utterance(utterance是对话中的语音片段，不一定是完整的句子)

该数据集庞大而多样，在语言形式、时间段、情感等方面有很大的变化。而我们的希望是，这种多样性使我们的模型，对多种形式的输入和查询具有鲁棒性。首先，我们将查看数据文件中部分行，以查看原始格式。

In [3]:
corpus_name = "cornell movie-dialogs corpus"
corpus = os.path.join("data", corpus_name)

def printLines(file, n=10):
    with open(file, 'rb') as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)

printLines(os.path.join(corpus, "movie_lines.txt"))

b'L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!\n'
b'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!\n'
b'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.\n'
b'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?\n'
b"L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.\n"
b'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow\n'
b"L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.\n"
b'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No\n'
b'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?\n'
b'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?\n'


### 数据处理

为了使用方便，我们会把原始数据处理成一个新的文件，这个新文件的每一行都是用TAB分割问题(query)和答案(response)对。为了实现这个目的，我们首先定义一些用于parsing原始文件 *movie_lines.txt*  的辅助函数。

 
-  ``loadLines`` 把*movie_lines.txt* 文件切分成 (lineID, characterID, movieID, character, text)
-  ``loadConversations`` 把上面的行group成一个个多轮的对话
-  ``extractSentencePairs`` 从上面的每个对话中抽取句对
将文件的每一行语料拆分都成一个字段字典，关键字是lineID、characterID、movieID、character和text，分别代表这一行的ID、人物ID、电影ID，人物名称和文本。最终输出一个字典，关键字是lineID，value是一个dict。value这个字典的关键字是lineID、characterID、movieID、character和text。




In [4]:
printLines(os.path.join(corpus, "movie_conversations.txt"))

b"u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']\n"
b"u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']\n"
b"u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']\n"
b"u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']\n"
b"u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']\n"
b"u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L271', 'L272', 'L273', 'L274', 'L275']\n"
b"u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L276', 'L277']\n"
b"u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L280', 'L281']\n"
b"u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L363', 'L364']\n"
b"u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L365', 'L366']\n"


In [5]:
# 把每一行都parse成一个dict，key是lineID、characterID、movieID、character和text
# 分别代表这一行的ID、人物ID、电影ID，人物名称和文本。
# 最终输出一个dict，key是lineID，value是一个dict。
# value这个dict的key是lineID、characterID、movieID、character和text
#将文件的每一行拆分为一个字段字典
def loadLines(fileName, fields):
    lines = {}
    with open(fileName, 'r', encoding='iso-8859-1') as f:
        for line in f:
            values = line.split(" +++$+++ ")
            # 抽取fields
            lineObj = {}
            for i, field in enumerate(fields):
                lineObj[field] = values[i]
            lines[lineObj['lineID']] = lineObj
    return lines


# 接下来我们根据movie_conversations.txt文件内容和以上输出的lines，把发言组成对话，
#最终输出一个列表。这个列表的每一个元素都是一个字典，关键字分别是character1ID、character2ID、movieID和utteranceIDs。
#分别表示这对话的第一个人物的ID，第二个的ID，电影的ID以及它包含的utteranceIDs。最后根据lines，
#，其数值是个列表，包含所有发言(以上得到的lines的值)。
def loadConversations(fileName, lines, fields):
    conversations = []
    with open(fileName, 'r', encoding='iso-8859-1') as f:
        for line in f:
            values = line.split(" +++$+++ ")
            # 提取字段
            convObj = {}
            for i, field in enumerate(fields):
                convObj[field] = values[i]
           # convObj["utteranceIDs"]是一个字符串，形如['L198', 'L199']
            #我们用eval把这个字符串变成一个字符串的列表。
            lineIds = eval(convObj["utteranceIDs"])
            # # 根据lineIds构造一个数组，根据lineId去lines里检索出存储utterance对象。
            convObj["lines"] = []
            for lineId in lineIds:
                convObj["lines"].append(lines[lineId])
            conversations.append(convObj)
    return conversations

#接下来从对话中抽取句对，假设一段对话包含s1,s2,s3,s4这4个发言
#那么就会返回3个句对：s1-s2,s2-s3和s3-s4。2-s3和s3-s4。

def extractSentencePairs(conversations):
    qa_pairs = []
    for conversation in conversations:
        # 遍历对话中的每一个句子，忽略最后一个句子，因为没有答案。
        for i in range(len(conversation["lines"]) - 1): 
            inputLine = conversation["lines"][i]["text"].strip()
            targetLine = conversation["lines"][i+1]["text"].strip()
            # 如果有空的句子就去掉 
            if inputLine and targetLine:
                qa_pairs.append([inputLine, targetLine])
    return qa_pairs

接下来我们利用上面的3个函数对原始数据进行处理，最终得到formatted_movie_lines.txt。

In [6]:
# 定义新的文件 
datafile = os.path.join(corpus, "formatted_movie_lines.txt")

delimiter = '\t'
# 对分隔符delimiter进行decode，这里对tab进行decode结果并没有变
delimiter = str(codecs.decode(delimiter, "unicode_escape"))

# 初始化dict lines，list conversations以及前面我们介绍过的field的id数组。
lines = {}
conversations = []
MOVIE_LINES_FIELDS = ["lineID", "characterID", "movieID", "character", "text"]
MOVIE_CONVERSATIONS_FIELDS = ["character1ID", "character2ID", "movieID", "utteranceIDs"]

# 首先使用loadLines函数处理movie_lines.txt 
print("\nProcessing corpus...")
lines = loadLines(os.path.join(corpus, "movie_lines.txt"), MOVIE_LINES_FIELDS)
# 接着使用loadConversations处理上一步的结果，得到conversations
print("\nLoading conversations...")
conversations = loadConversations(os.path.join(corpus, "movie_conversations.txt"),
                                  lines, MOVIE_CONVERSATIONS_FIELDS)

# 输出到一个新的csv文件
print("\nWriting newly formatted file...")
with open(datafile, 'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
    # 使用extractSentencePairs从conversations里抽取句对。
    for pair in extractSentencePairs(conversations):
        writer.writerow(pair)

# 输出一些行用于检查 
print("\nSample lines from file:")
printLines(datafile)


Processing corpus...

Loading conversations...

Writing newly formatted file...

Sample lines from file:
b"Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\tWell, I thought we'd start with pronunciation, if that's okay with you.\r\n"
b"Well, I thought we'd start with pronunciation, if that's okay with you.\tNot the hacking and gagging and spitting part.  Please.\r\n"
b"Not the hacking and gagging and spitting part.  Please.\tOkay... then how 'bout we try out some French cuisine.  Saturday?  Night?\r\n"
b"You're asking me out.  That's so cute. What's your name again?\tForget it.\r\n"
b"No, no, it's my fault -- we didn't have a proper introduction ---\tCameron.\r\n"
b"Cameron.\tThe thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\r\n"
b"The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister. 

### 创建词典


接下来我们需要构建词典然后把问答句对加载到内存里。我们的输入是一个句对，每个句子都是词的序列，因为机器学习只能处理数值，因此我们需要建立词到数字ID的映射。为此，我们会定义一个Voc类，它会保存词到ID的映射，同时也保存反向的从ID到词的映射。
除此之外，它还记录每个词出现的次数，以及总共出现的词的个数。这个类提供addWord方法来增加一个词，用addSentence方法来增加句子，也提供trim方法来去除低频的词。




In [7]:
# 预定义的token
PAD_token = 0  # 表示填槽 
SOS_token = 1  # 句子的开始 
EOS_token = 2  # 句子的结束 

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # 目前有SOS, EOS, PAD这3个token。

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # 删除频次小于min_count的token 
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # 重新构造词典 
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens
        
        # 重新构造后词频就没有意义了(都是1)
        for word in keep_words:
            self.addWord(word)

有了上面的Voc类我们就可以通过问答句对来构建词典了。但是在构建之前我们需要进行一些预处理。
首先我们需要使用函数unicodeToAscii来把unicode字符变成ascii，比如把à变成a。
注意，这里的代码只是用于处理西方文字，如果是中文，这个函数直接会丢弃掉。
接下来把所有字母变成小写同时丢弃掉字母和常见标点(.!?)之外的所有字符。
最后为了训练收敛，我们会用函数filterPairs去掉长度超过MAX_LENGTH的句子(句对)。


In [8]:
MAX_LENGTH = 10  # 句子最大长度是10个词(包括EOS等特殊词)

# 把Unicode字符串变成ASCII
# 参考https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )
 
def normalizeString(s):
    # 变成小写、去掉前后空格，然后unicode变成ascii
    s = unicodeToAscii(s.lower().strip())
    # 在标点前增加空格，这样把标点当成一个词
    s = re.sub(r"([.!?])", r" \1", s)
    # 字母和标点之外的字符都变成空格
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    # 因为把不用的字符都变成空格，所以可能存在多个连续空格
    # 下面的正则替换把多个空格变成一个空格，最后去掉前后空格
    s = re.sub(r"\s+", r" ", s).strip()
    return s

# 读取问答句对并且返回Voc词典对象 
def readVocs(datafile, corpus_name):
    print("Reading lines...")
    # 文件每行读取到list lines中。 
    lines = open(datafile, encoding='utf-8').\
        read().strip().split('\n')
    # 每行用tab切分成问答两个句子，然后调用normalizeString函数进行处理。
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    voc = Voc(corpus_name)
    return voc, pairs

def filterPair(p): 
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

# 过滤太长的句对 
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

# 使用上面的函数进行处理，返回Voc对象和句对的list 
def loadPrepareData(corpus, corpus_name, datafile):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs


# 装载 voc 和句子对
save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus, corpus_name, datafile)
# 输出一些句对
print("\npairs:")
for pair in pairs[:10]:
    print(pair)

Start preparing training data ...
Reading lines...
Read 221282 sentence pairs
Trimmed to 64271 sentence pairs
Counting words...
Counted words: 18008

pairs:
['there .', 'where ?']
['you have my word . as a gentleman', 'you re sweet .']
['hi .', 'looks like things worked out tonight huh ?']
['you know chastity ?', 'i believe we share an art instructor']
['have fun tonight ?', 'tons']
['well no . . .', 'then that s all you had to say .']
['then that s all you had to say .', 'but']
['but', 'you always been this selfish ?']
['do you listen to this crap ?', 'what crap ?']
['what good stuff ?', 'the real you .']


为了收敛更快，我们可以去除掉一些低频词。这可以分为两步：
1) 使用voc.trim函数去掉频次低于MIN_COUNT 的词。
2) 去掉包含低频词的句子。



In [9]:
MIN_COUNT = 3    # 阈值为3


def trimRareWords(voc, pairs, MIN_COUNT):
    # 去掉voc中频次小于3的词 
    voc.trim(MIN_COUNT)
    # 保留的句对 
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        # 检查问题
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        # 检查答案
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break

        # 如果问题和答案都只包含高频词，我们才保留这个句对
        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs


# 实际进行处理
pairs = trimRareWords(voc, pairs, MIN_COUNT)

keep_words 7823 / 18005 = 0.4345
Trimmed from 64271 pairs to 53165, 0.8272 of total


### 为模型准备数据

前面我们构建了词典，并且对训练数据进行预处理并且滤掉一些句对，但是模型最终用到的是Tensor。最简单的办法是一次处理一个句对，那么上面得到的句对直接就可以使用。但是为了加快训练速度，尤其是重复利用GPU的并行能力，我们需要一次处理一个batch的数据。

对于某些问题，比如图像来说，输入可能是固定大小的(或者通过预处理缩放成固定大小），但是对于文本来说，我们很难把一个二十个词的句子"缩放"成十个词同时还保持语义不变。但是为了充分利用GPU等计算自由，我们又必须变成固定大小的Tensor，因此我们通常会使用Padding的技巧，把短的句子补充上零使得输入大小是(batch, max_length)，这样通过一次就能实现一个batch数据的forward或者backward计算。当然padding的部分的结果是没有意义的，比如某个句子实际长度是5，而max_length是10，那么最终forward的输出应该是第5个时刻的输出，后面5个时刻计算是无用功。方向计算梯度的时候也是类似的，我们需要从第5个时刻开始反向计算梯度。为了提高效率，通常把长度接近的训练数据放到一个batch里面，这样无用的计算是最少的。因此我们通常把全部训练数据根据长度划分成一些组，比如长度小于4的一组，长度4到8的一组，长度8到12的一组，...。然后每次随机的选择一个组，再随机的从一组里选择batch个数据。不过本教程并没有这么做，而是每次随机的从所有pair里随机选择batch个数据。

原始的输入通常是batch个list，表示batch个句子，因此自然的表示方法为(batch, max_length)，这种表示方法第一维是batch，每移动一个下标得到的是一个样本的max_length个词(包括padding)。因为RNN的依赖关系，我们在计算t+1时刻必须知道t时刻的结果，因此我们无法用多个核同时计算一个样本的forward。但是不同样本之间是没有依赖关系的，因此我们可以在根据t时刻batch样本的当前状态计算batch个样本的输出和新状态，然后再计算t+2时刻，...。为了便于GPU一次取出t时刻的batch个数据，我们通常把输入从(batch, max_length)变成(max_length, batch)，这样使得t时刻的batch个数据在内存(显存)中是连续的，从而读取效率更高。这个过程如下图所示，原始输入的大小是(batch=6, max_length=4)，转置之后变成(4,6)。这样某个时刻的6个样本数据在内存中是连续的。

![](seq2seq_batches.png)

 
因此我们会用一些工具函数来实现上述处理。

``inputVar``函数把batch个句子padding后变成一个LongTensor，大小是(max_length, batch)，同时会返回一个大小是batch的list lengths，说明每个句子的实际长度，这个参数后面会传给PyTorch，从而在forward和backward计算的时候使用实际的长度。

``outputVar``函数和``inputVar``类似，但是它输出的第二个参数不是lengths，而是一个大小为(max_length, batch)的mask矩阵(tensor)，某位是0表示这个位置是padding，1表示不是padding，这样做的目的是后面计算方便。当然这两种表示是等价的，只不过lengths表示更加紧凑，但是计算起来不同方便，而mask矩阵和outputVar直接相乘就可以把padding的位置给mask(变成0)掉，这在计算loss时会非常方便。

``batch2TrainData`` 则利用上面的两个函数把一个batch的句对处理成合适的输入和输出Tensor。




In [10]:

def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]
#合并数据，相当于行列转置

def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

# 记录 PAD_token的位置为0，其他的为1

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# 返回填充前(加入结束index EOS_token做标记)的长度和填充后的输入序列张量

def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# 返回填充目标序列张量、填充掩码和最大目标长度
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.ByteTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# 返回给定batch对的所有项目
def batch2TrainData(voc, pair_batch):
    
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len


# 验证例子
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[2463,   33,   51,   77, 2266],
        [  66, 2387, 2779,   37,  164],
        [   9,   36,  682,   36,    4],
        [6257,   37,    4,    6,    2],
        [   4,   12, 1982,    2,    0],
        [   4, 5620,    4,    0,    0],
        [   4, 1557,    2,    0,    0],
        [   2,    2,    0,    0,    0]])
lengths: tensor([8, 8, 7, 5, 4])
target_variable: tensor([[ 247,  651,  112,   76,   25],
        [ 117,  598,   77,   37,  200],
        [   7,   76,  115,  112,  483],
        [  24,    4,  159,  180, 2267],
        [   6,    2,   94,   56,    4],
        [   2,    0,    7,    9,   25],
        [   0,    0,  141, 4997,  387],
        [   0,    0,   83, 5451,    4],
        [   0,    0,    6,    4,    2],
        [   0,    0,    2,    2,    0]])
mask: tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 0, 1, 1, 1],
        [0, 0, 1, 1, 1],
        [0, 0, 1, 1, 1],
      

### (1)编码器
我们聊天机器人是一个序列到序列(seq2seq)模型。seq2seq模型的目标是将变长序列作为输入，并使用固定大小的模型返回变长序列作为输出。
研究发现，通过将两个独立的RNN循环神经网络结合起来，可以完成这一任务。一个RNN充当编码器，它将可变长度的输入序列编码为固定长度的上下文向量。理论上，这个上下文向量(RNN的最后一个隐含层)将包含输入机器人的查询句子的语义信息。第二个RNN是一个解码器，它接受一个输入单词和上下文向量，并返回序列中下一个单词的概率和一个在下一次迭代中使用的隐藏状态。
我们的编码器的核心是由Kyunghyun Cho, Dzmitry Bahdanau, Fethi Bougares Holger Schwenk, Yoshua Bengio(京铉町、德米特里·巴达瑙、费蒂·布加勒斯·霍尔格·施文克、约书亚·本吉奥).等人发明的多层门循环单元。在2014年，使用 GRU的双向变体，这意味着基本上有两个独立的RNN：一个以正常的顺序输入输入序列，另一个以相反的顺序输入输入序列。每个网络的输出在 每个时间步骤求和。使用双向GRU将为我们提供编码过去和未来上下文的优势。(注意:embedding层用于在任意大小的特征空间中对我们的单词索引进行编码。对于我们的模型，此图层会将每个单词映射到大小为 hidden_size的特征空间。训练后，这些值会被编码成和他们相似的有意义词语。)
最后，如果将填充的一批序列传递给RNN模块，我们必须分别使用torch.nn.utils.rnn.pack_padded_sequence和torch.nn.utils.rnn.pad_packed_sequence 在RNN传递时分别进行填充和反填充。
计算流程图为：
（1）将单词索引转换为词嵌入 embeddings。
（2）为RNN模块打包填充batch序列。
（3）通过GRU进行前向传播。
（4）反填充。
（5）对双向GRU输出求和。
（6）返回输出和最终隐藏状态。
输入：
input_seq：一批输入句子; shape =(max_length，batch_size)
input_lengths：batch中每个句子对应的句子长度列表;shape=(batch_size)
hidden:隐藏状态;shape =(n_layers x num_directions，batch_size，hidden_size)
输出：
outputs：GRU最后一个隐藏层的输出特征(双向输出之和);
shape =(max_length，batch_size，hidden_size)
hidden：从GRU更新隐藏状态;shape =(n_layers x num_directions，batch_size，hidden_size)
class EncoderRNN(nn.Module):



In [11]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # 初始化GRU; input_size和hidden_size参数都设置为'hidden_size'
    # 因为我们的输入大小是一个嵌入了多个特征的单词==hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
       # 将单词索引转换为词向量
        embedded = self.embedding(input_seq)
         # 为RNN模块填充一批batch序列
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
       # 正向通过GRU
        outputs, hidden = self.gru(packed, hidden)
         # 打开填充
        outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(outputs)
        # 双向GRU输出总和
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # 返回输出和最终隐藏状态
        return outputs, hidden

### (2)解码器
解码器RNN以token-by-token(令牌传递)的方式生成响应语句。它使用编码器的上下文向量和内部隐藏状态来生成序列中的下一个单词。它持续生成单词，直到输出是EOS_token，这个表示句子的结尾。一个 vanilla seq2seq 解码器的常见问题是，如果我们只依赖于上下文向量来编码整个输入序列的含义，那么我们很可能会丢失信息。尤其是在处理长输入序列时，这极大地限制了我们的解码器的能力。
为了解决这个问题，Dzmitry Bahdanau(德米特里·巴达诺).等人创建了一种“attention mechanism ”，允许解码器关注输入序列的某些部分，而不是在每一步都使用完全固定的上下文。
在一个高的层级中，用解码器的当前隐藏状态和编码器输出来计算注意力。输出注意力的权重与输入序列具有相同的大小，允许我们将它们乘以编码器输出，给出一个加权和，表示要注意的编码器输出部分。
Minh-Thang Luong, (明成 隆).等人通过创造“Global attention ”，改善了Bahdanau et al.的基础工作。关键的区别在于，对于“Global attention ”，我们考虑所有编码器的隐藏状态，而不是Dzmitry Bahdanau(德米特里·巴达诺)等人的“Local attention ”，它只考虑当前步中编码器的隐藏状态。另一个区别在于，通过“Global attention ”，我们仅使用当前步的解码器的隐藏状态来计算注意力权重。Dzmitry Bahdanau(德米特里·巴达诺)等人的注意力计算需要知道前一步中解码器的状态。 此外，Minh-Thang Luong(明成良).等人提供各种方法来计算编码器输出和解码器输出 之间的注意权重(能量)，称之为“score functions ”。
总体而言，Global attention机制可以通过下述的流程图进行总结。请注意，我们将“Attention Layer ”用一个名为Attn的nn.Module来单独实现。 
该模块的输出是经过softmax标准化后权重张量的大小(batch_size，1，max_length)。

 




In [12]:
# Luong 注意力layer
class Attn(torch.nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = torch.nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = torch.nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = torch.nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
       
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)
    
    # 根据给定的方法计算注意力(能量)  
    def forward(self, hidden, encoder_outputs):
        
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
           
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # 转置max_length和batch_size尺寸
        attn_energies = attn_energies.t()

        # 返回softmax归一化概率得分
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

现在我们已经定义了注意力子模块，我们可以实现真实的解码器模型。对于解码器，我们将每次手动进行一批次的输入。这意味着我们的词嵌入张量和GRU输出都将具有相同大小(1，batch_size，hidden_size)。
计算流程图：
①　获取当前输入的词嵌入
②　通过单向GRU进行前向传播
③　通过2输出的当前GRU计算注意力权重
④　将注意力权重乘以编码器输出以获得新的“weighted sum ”上下文向量
⑤　使用Minh-Thang Luong, (明成 隆)公式5.(https://arxiv.org/pdf/1508.04025v3.pdf)连接加权上下文向量和GRU输出
⑥　使用Minh-Thang Luong, (明成 隆)公式6.(https://arxiv.org/pdf/1508.04025v3.pdf)预测下一个单词
⑦　返回输出和最终隐藏状态
输入：
input_step：每一步输入序列batch(一个单词);shape =(1，batch_size)
last_hidden：GRU的最终隐藏层;shape =(n_layers x num_directions，batch_size，hidden_size)
encoder_outputs：编码器模型的输出;shape =(max_length，batch_size，hidden_size)
输出：
output: 一个softmax标准化后的张量， 代表了每个单词在解码序列中是下一个输出单词的概率;shape =(batch_size，voc.num_words)
hidden: GRU的最终隐藏状态;
shape =(n_layers x num_directions，batch_size，hidden_size)




In [13]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # 定义层
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # 注意：我们一次运行一个步骤(单词)
        # 获取当前输入字的嵌入
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # 通过单向GRU转发
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # 从当前GRU输出计算注意力
        attn_weights = self.attn(rnn_output, encoder_outputs)
        
        # 将注意力权重乘以编码器输出以获得新的“加权和 ”上下文向量
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
         # 使用Minh-Thang Luong, (明成 隆)等的公式5连接加权上下文向量和GRU输出
        rnn_output = rnn_output.squeeze(0)
        # context从(64, 1, 500)变成(64, 500)
        context = context.squeeze(1)
        # 拼接得到(64, 1000)
        concat_input = torch.cat((rnn_output, context), 1)
        # self.concat是一个矩阵(1000, 500)，
        # self.concat(concat_input)的输出是(64, 500)
        # 然后用tanh把输出返回变成(-1,1)，concat_output的shape是(64, 500)
        concat_output = torch.tanh(self.concat(concat_input))

         # 使用Minh-Thang Luong, (明成 隆)等的公式6预测下一个单词
        output = self.out(concat_output)
        # 用softmax变成概率，表示当前时刻输出每个词的概率。
        output = F.softmax(output, dim=1)
        # 返回输出和在最终隐藏状态
        return output, hidden

## 定义训练步骤

由于我们处理的是批量填充序列，因此在计算损失时我们不能简单地考虑张量的所有元素。我们定义maskNLLLoss可以根据解码器的输出张量、 描述目标张量填充的binary mask张量来计算损失。该损失函数计算与mask tensor中的1对应的元素的平均负对数似然。




In [14]:
def maskNLLLoss(inp, target, mask):
    # 计算实际的词的个数，因为padding是0，非padding是1，因此sum就可以得到词的个数
    nTotal = mask.sum()
    
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()


### 定义训练迭代


PyTorch的RNN模块(RNN，LSTM，GRU)可以像任何其他非重复层一样使用，只需将整个输入序列(或一批序列)传递给它们。我们在编码器中使用GRU层就是这样的。实际情况是，在计算中有一个迭代过程，进行循环计算隐藏状态的每一步。或者每次只运行一个模块，在这种情况下，我们在训练过程中手动循环遍历序列，就像我们必须为解码器模型做的那样。只要你正确的维护这些模型的模块，就可以非常简单的实现训练模型。
  





In [15]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    # 梯度清空
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # 设置device，从而支持GPU，当然如果没有GPU也能工作。
    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    # 初始化变量
    loss = 0
    print_losses = []
    n_totals = 0

    # 正向传递编码器
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # 创建初始解码器输入(从每个句子的SOS令牌开始)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # 将初始解码器隐藏状态设置为编码器的最终隐藏状态
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # 确定我们是否此次迭代使用teacher forcing
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # 通过解码器一次一步地转发一批次序列
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: 下一个输入是当前的目标
            decoder_input = target_variable[t].view(1, -1)
            # 计算并累计损失
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            #下一个输入是解码器自己的当前输出
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # 计算并累计损失
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # 执行反向传播
    loss.backward()

    # 对encoder和decoder进行梯度裁剪
    _ = torch.nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = torch.nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # 更新参数
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

### 训练迭代过程


现在终于将完整的训练步骤与数据结合在一起了。给定传递的模型、优化器、数据等，trainIters函数负责运行n_iterations的训练。 这个功能显而易见，因为我们通过train函数的完成了繁重工作。
需要注意的一点是，当我们保存模型时，我们会保存一个包含编码器和解码器state_dicts(参数)、优化器的state_dicts、损失、迭代等的压缩包。以这种方式保存模型将为我们checkpoint,提供最大的灵活性。
加载checkpoint后，我们将能够使用模型参数进行推理，或者我们可以在我们中断的地方继续训练。


In [16]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # 随机选择n_iteration个batch的数据(pair)
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # # 初始化
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    #  训练循环
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        #从batch中提取字段
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # 训练一个batch的数据
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # 打印进度
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # 保存checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

## 效果测试

在训练模型后，我们希望能够自己与机器人交谈。首先，我们必须定义我们希望模型如何解码编码输入。
贪婪解码是我们在不使用 teacher forcing(使用来自先验时间步长的输出作为输入) 时在训练期间使用的解码方法。换句话说，对于每一步，我们只需从具有最高 softmax 值的 decoder_output 中选择单词。该解码方法在单步长级别上是最佳的。
为了便于贪婪解码操作，我们定义了一个GreedySearchDecoder类。当运行时，类的实例化对象输入序列(input_seq)的大小是(input_seq length，1)， 标量输入(input_length)长度的张量和 max_length 来约束响应句子长度。使用以下计算流程图来评估输入句子：
计算流程图
(1)通过编码器模型前向计算。
(2)准备编码器的最终隐藏层，作为解码器的第一个隐藏输入。
(3)将解码器的第一个输入初始化为 SOS_token。
(4)将初始化张量追加到解码后的单词中。
(5)一次迭代解码一个单词token：
①　通过解码器进行前向计算。
②　获得最可能的单词token及其softmax分数。
③　记录token和分数。
④　准备当前token作为下一个解码器的输入。
(6)返回收集到的词 tokens 和分数。
   



In [17]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # 通过编码器模型转发输入
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # 将编码器的最终隐藏层准备为解码器的第一个隐藏输入
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        #使用SOS_token初始化解码器输入
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # 初始化要附加已解码单词的张量
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # 循环，这里只使用长度限制，后面处理的时候把EOS去掉了。
        for _ in range(max_length):
            # 正向通过解码器
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
           # 获得最可能的单词标记及其softmax分数
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # 把解码结果保存到all_tokens和all_scores里
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # 准备当前令牌作为下一个解码器输入(添加维度)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # 返回收集到的词tokens和分数
        return all_tokens, all_scores

### 评估文本


我们已经定义了解码方法，可以编写用于评估字符串输入句子的函数。evaluate函数管理输入句子的低层级处理过程。我们首先使 用batch_size == 1将句子格式化为输入batch的单词索引。我们通过将句子的单词转换为相应的索引，并通过转换维度来为我们的模型准备张量。我们还创建了一个lengths张量，其中包含输入句子的长度。在这种情况下，lengths是标量，因为我们一次只评估一个句子(batch_size == 1)。接下来，我们使用我们的GreedySearchDecoder实例化后的对象(searcher)获得解码响应句子的张量。最后，我们将响应的索引转换为单 词并返回已解码单词的列表。
evaluateInput充当聊天机器人的用户接口。调用时，将生成一个输入文本字段，我们可以在其中输入查询语句。在输入我们的输入句子并 按 Enter 后，我们的文本以与训练数据相同的方式标准化，并最终被输入到评估函数以获得解码的输出句子。我们循环这个过程，这样我们可 以继续与我们的机器人聊天直到我们输入“q ”或“quit ”。
最后，如果输入的句子包含一个不在词汇表中的单词，我们会通过打印错误消息并提示用户输入另一个句子来优雅地处理。




In [18]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### 格式化输入句子作为batch
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # 创建lengths张量
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # 转置batch的维度以匹配模型的期望
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # 放到合适的设备上(比如GPU)
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # 用searcher解码句子
    tokens, scores = searcher(input_batch, lengths, max_length)
    # ID变成词。
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # 获取输入句子
            input_sentence = input('> ')
            # 检查是否退出
            if input_sentence == 'q' or input_sentence == 'quit': break
            # 句子归一化
            input_sentence = normalizeString(input_sentence)
            # 格式化和打印回复句
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # 去掉EOS后面的内容
            words = []
            for word in output_words:
                if word == 'EOS':
                    break
                elif word != 'PAD':
                    words.append(word)
            print('Bot:', ' '.join(words))

        except KeyError:
            print("Error: Encountered unknown word.")

## 训练和测试模型

最后，我们开始运行模型。无论我们是否想要训练或测试聊天机器人模型，我们都必须初始化各个编码器和解码器模型。在接下来的部分，我们设置了所需的配置，选择从头开始，或者设置加载的检查点，并构建和初始化模型。您可以随意使用不同的模型配置来优化性能。




In [19]:
# 配置模型
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

# 设置检查点以加载; 如果从头开始，则设置为None。
loadFilename = None
checkpoint_iter = 2
  

# 如果提供了loadFilename，则加载模型
if loadFilename:
    # 如果在同一台机器上加载，则对模型进行训练
    checkpoint = torch.load(loadFilename)
    # 否则比如checkpoint是在GPU上得到的，但是我们现在又用CPU来训练或者测试，那么注释掉下面的代码
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# 初始化词向量
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# 初始化编码器和解码器模型
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# 使用合适的设备
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


## 模型训练
如果要训练模型，请运行以下部分。
首先我们设置训练参数，然后初始化我们的优化器，最后我们调用trainIters函数来运行我们的训练迭代。





In [20]:
# 配置训练的超参数和优化器 
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 100
print_every = 1
save_every = 500

# 确保dropout layers在训练模型中
encoder.train()
decoder.train()

# 初始化优化器 
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# 开始训练
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Training!
Initializing ...
Training...


C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: masked_select received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  ..\aten\src\ATen\native\TensorAdvancedIndexing.cpp:733.)
  
C:\Users\Administrator\Anaconda3\lib\site-packages\torch\autograd\__init__.py:132: UserWarning: masked_scatter_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  ..\aten\src\ATen\native\LegacyDefinitions.cpp:20.)
  allow_unreachable=True)  # allow_unreachable flag


Iteration: 1; Percent complete: 1.0%; Average loss: 8.9668
Iteration: 2; Percent complete: 2.0%; Average loss: 8.8601
Iteration: 3; Percent complete: 3.0%; Average loss: 8.7393
Iteration: 4; Percent complete: 4.0%; Average loss: 8.4241
Iteration: 5; Percent complete: 5.0%; Average loss: 8.1144
Iteration: 6; Percent complete: 6.0%; Average loss: 7.6378
Iteration: 7; Percent complete: 7.0%; Average loss: 6.9383
Iteration: 8; Percent complete: 8.0%; Average loss: 6.9917
Iteration: 9; Percent complete: 9.0%; Average loss: 6.5861
Iteration: 10; Percent complete: 10.0%; Average loss: 6.6975
Iteration: 11; Percent complete: 11.0%; Average loss: 6.4147
Iteration: 12; Percent complete: 12.0%; Average loss: 6.1059
Iteration: 13; Percent complete: 13.0%; Average loss: 5.7933
Iteration: 14; Percent complete: 14.0%; Average loss: 5.5153
Iteration: 15; Percent complete: 15.0%; Average loss: 5.5470
Iteration: 16; Percent complete: 16.0%; Average loss: 5.4658
Iteration: 17; Percent complete: 17.0%; Av

### 测试


运行如下代码，开始与您的模型进行聊天了。同时，可以尝试通过调整模型和训练参数以及 自定义训练模型的数据来定制聊天机器人的行为。




In [21]:
# 将dropout layers设置为eval模式 
encoder.eval()
decoder.eval()

# 构造searcher对象 
searcher = GreedySearchDecoder(encoder, decoder)

# 测试
evaluateInput(encoder, decoder, searcher, voc)

> hello?
Bot: i you .
> how are you doing?
Bot: i i you .
> q


## 结论

上面介绍了怎么从零开始训练一个chatbot，读者可以用自己的数据训练一个chatbot试试，看看能不能用来解决一些实际业务问题。


